# Artificial Neural Networks (supervised)
In this section, we will train an Artificial Neural network with the same input as we trained the KNN algorithm. We start by generating the signals and the database.

In [1]:
import numpy as np
import numpy.typing as npt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from numpy.random import MT19937, RandomState, SeedSequence
from ipywidgets import widgets
from IPython.display import display, Audio
import matplotlib.pyplot as plt

random_state = 123456789
rs = RandomState(MT19937(SeedSequence(random_state)))
ws = {'description_width': 'initial'}
sample_rate = widgets.IntSlider(40000, min=10000, max=50000, description="Sample frequency (hZ)", style=ws)
display(sample_rate)

IntSlider(value=40000, description='Sample frequency (hZ)', max=50000, min=10000, style=SliderStyle(descriptio…

In [2]:
def generate_noisy_signal(
    sample_frequency: int,
    amplitudes: npt.NDArray[np.float64],
    frequencies: npt.NDArray[np.float64],
    phases: npt.NDArray[np.float64], noise: float=1, T:float=2.):
    N = int(T*sample_frequency)
    t = np.linspace(0, T, N, endpoint=False)
    assert len(frequencies) == len(phases)
    assert len(amplitudes) == len(phases)
    n = noise*np.random.randn(len(t))
    return t, n+sum([A*np.cos(2*np.pi*f*t+p) for (A,f,p) in zip(amplitudes, frequencies, phases)])

In [3]:
def generate_database(num_samples:int, T:float, num_windows:int, high:int=3, low:int=7, threshold:int=8e3):
    A = rs.uniform(-4, 4, size=num_samples)
    phase = np.pi * rs.random(num_samples)
    noise = rs.uniform(-2, 2, size=num_samples) * rs.random_integers(0,1, size=num_samples)
    frequencies = rs.uniform(0, sample_rate.value//2, size=num_samples) 
    features = np.zeros((num_samples, num_windows))
    labels = np.zeros(num_samples, dtype=np.int32)
    for i in range(num_samples):
        _, signal = generate_noisy_signal(sample_rate.value, A[i:i+1], frequencies[i:i+1],
                                          phase[i:i+1],noise[i:i+1], T)
        features[i] = extract_features(signal, num_windows, sample_rate.value)
        labels[i] = high if frequencies[i]>threshold else low
    target_map = {low:"low", high:"high"}
    feature_names = [f"Signal amplitude integrated in frequency window {i}" for i in range(num_windows)]

    return features, labels, feature_names, target_map

def extract_features(signal:npt.NDArray[np.float64], N:int, sampling_frequency:int):
    rfft = np.fft.rfft(signal)
    f_per_band = len(rfft)//N
    rem = len(rfft) % N
    freq_per_band = np.full(N, f_per_band, dtype=np.int32)
    freq_per_band[:rem]+=1
    offsets= np.zeros(N+1, dtype=np.int32)
    offsets[1:] = np.cumsum(freq_per_band)
    RMS_amplitude = np.zeros(N, dtype=np.float64)
    mag = 1/len(signal) * np.abs(rfft)
    for i in range(N):
        RMS_amplitude[i] = np.sqrt(1/(offsets[i+1]-offsets[i])*np.sum(mag[offsets[i]:offsets[i+1]]**2))
    return RMS_amplitude

T = 1
num_samples = 1000
num_windows = 25
features, labels, feature_names, target_map = generate_database(T=T, num_samples=num_samples, num_windows=num_windows)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=random_state)


/tmp/ipykernel_88337/3827608889.py:4: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  noise = rs.uniform(-2, 2, size=num_samples) * rs.random_integers(0,1, size=num_samples)


In [12]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
@widgets.interact(
    K=widgets.RadioButtons(options=[1,2,4,8], value=1,
                           description='Number of hidden layers:',
                           disabled=False, style=ws),
    method=widgets.Select(options=['sgd', "adam", "lbfgs"], value='sgd',
                           description='Method for weight optimization:',
                           disabled=False, style=ws),
    activation=widgets.Select(options=["relu", "identity", "tanh", "logistic"], value='logistic',
                           description='Activation function:',
                           disabled=False, style=ws),
    rate=widgets.FloatLogSlider(value=0.01, base=10, min=-5, max=0, step=0.5,
                                description="Initial learning rate", style=ws),
    neurons=widgets.RadioButtons(options=[1,10,100,200,400], value=10,
                                description="Neurons per layer", style=ws),
    verbose=widgets.Checkbox(value=False, description='Print training progress', disabled=False)
)
def test_and_train(method:str, K:int, activation:str, rate:float, neurons:int, verbose:bool):
    target_labels = list(target_map.keys())
    
    hidden_layers = (neurons, ) * K
    clf = MLPClassifier(solver=method, learning_rate_init=rate,
                        random_state=random_state, hidden_layer_sizes=hidden_layers,
                        activation=activation, verbose=verbose, learning_rate="adaptive")
    clf.fit(X_train, y_train)

    predicted_train = clf.predict(X_train)
    predicted_test = clf.predict(X_test)
    print(f"Train accuracy: {accuracy_score(y_train, predicted_train)}, {accuracy_score(y_train, predicted_train, normalize=False)}/{len(y_train)} correct")
    print(f"Test accuracy: {accuracy_score(y_test, predicted_test)}, {accuracy_score(y_test, predicted_test, normalize=False)}/{len(y_test)} correct")
    disp = ConfusionMatrixDisplay(confusion_matrix(predicted_test,y_test))
    disp.plot()

interactive(children=(Select(description='Method for weight optimization:', options=('sgd', 'adam', 'lbfgs'), …